In [1]:
import os, sys
import pandas as pd
import numpy as np

SEQ ALIGMENT (manual, no avail tools applied) ==>> NOT USED

In [2]:
template = 'AYVLLIMFWDKRW'
seq = 'ALWYLMWDK'

print(template,len(template))
print(seq, len(seq))

t_mod = template
s_mod=''
match_list = {}
for s in seq:
    for t in template:
        if s==t:
            match_pos = template.index(t) + 1
            resi = seq.index(s) + 1
            if match_pos not in match_list.keys():
                match_list[match_pos] = (s,resi)
                
            break

sorted_list = sorted(match_list)
#print('\n',match_list,'\n',sorted_list)

for i in range(len(template)):
    if i+1 in sorted_list:
        s_mod += match_list[i+1][0]
        if match_list[i+1][1] > i+1: 
            t_mod = template[:i] + '-' + template[i:]
            s_mod = s_mod[:i] + seq[i] + s_mod[i:]
    else:
        s_mod += '-'
    

print('\naligned:\n',t_mod,'\n',s_mod)

AYVLLIMFWDKRW 13
ALWYLMWDK 9

aligned:
 A-YVLLIMFWDKRW 
 ALY-L--M-WDK--


In [3]:
# alignment with same lengths
def align_sameLen(template,seq_list=[seq]):
    print(template,'\n')
    prev_score = 0
    for seq in seq_list:
        score = 0
        s_mod=''
        for t,s in zip(list(template),list(seq)):
            if t==s: 
                s_mod += s
                score += 1
            else: s_mod += '-'
        score = score / len(template)
        max_score = max(score,prev_score)
        prev_score = score
        print(f'{s_mod}  score: {score:.3f}\n')
    return template, max_score


In [4]:
template = 'LYYGYGYWYFDV'
seq = ['RDYDYAGRYFDY','RDYDYAGRGFDY']
align_sameLen(template,seq)

LYYGYGYWYFDV 

--Y-Y---YFD-  score: 0.417

--Y-Y----FD-  score: 0.333



('LYYGYGYWYFDV', 0.4166666666666667)

In [2]:
# BLOSUM62 MATRIX
# using gap score from mafft
BLOSUM62 = {
    ('C','C'): 9, ('C','S'):-1, ('C','T'):-1, ('C','A'): 0, ('C','G'):-3, ('C','P'):-3, ('C','D'):-3, ('C','E'):-4, ('C','Q'):-3, ('C','N'):-3, ('C','H'):-3, ('C','R'):-3, ('C','K'):-3, ('C','M'):-1, ('C','I'):-1, ('C','L'):-1, ('C','V'):-1, ('C','W'):-2, ('C','Y'):-2, ('C','F'):-2, ('C','-'):-1.53,
    ('S','C'):-1, ('S','S'): 4, ('S','T'): 1, ('S','A'): 1, ('S','G'): 0, ('S','P'):-1, ('S','D'): 0, ('S','E'): 0, ('S','Q'): 0, ('S','N'): 1, ('S','H'):-1, ('S','R'):-1, ('S','K'): 0, ('S','M'):-1, ('S','I'):-2, ('S','L'):-2, ('S','V'):-2, ('S','W'):-3, ('S','Y'):-2, ('S','F'):-2, ('S','-'):-1.53,
    ('T','C'):-1, ('T','S'): 1, ('T','T'): 5, ('T','A'): 0, ('T','G'):-2, ('T','P'):-1, ('T','D'):-1, ('T','E'):-1, ('T','Q'):-1, ('T','N'): 0, ('T','H'):-2, ('T','R'):-1, ('T','K'):-1, ('T','M'):-1, ('T','I'):-1, ('T','L'):-1, ('T','V'):-1, ('T','W'):-2, ('T','Y'):-2, ('T','F'):-2, ('T','-'):-1.53,
    ('A','C'): 0, ('A','S'): 1, ('A','T'): 0, ('A','A'): 4, ('A','G'): 0, ('A','P'):-1, ('A','D'):-2, ('A','E'):-1, ('A','Q'):-1, ('A','N'):-2, ('A','H'):-2, ('A','R'):-1, ('A','K'):-1, ('A','M'):-1, ('A','I'):-1, ('A','L'):-1, ('A','V'):-1, ('A','W'):-3, ('A','Y'):-2, ('A','F'):-2, ('A','-'):-1.53,
    ('G','C'):-3, ('G','S'): 0, ('G','T'):-2, ('G','A'): 0, ('G','G'): 6, ('G','P'):-2, ('G','D'):-1, ('G','E'):-2, ('G','Q'):-2, ('G','N'): 0, ('G','H'):-2, ('G','R'):-2, ('G','K'):-2, ('G','M'):-3, ('G','I'):-4, ('G','L'):-4, ('G','V'):-3, ('G','W'):-2, ('G','Y'):-3, ('G','F'):-3, ('G','-'):-1.53,
    ('P','C'):-3, ('P','S'):-1, ('P','T'):-1, ('P','A'):-1, ('P','G'):-2, ('P','P'): 7, ('P','D'):-1, ('P','E'):-1, ('P','Q'):-1, ('P','N'):-2, ('P','H'):-2, ('P','R'):-2, ('P','K'):-1, ('P','M'):-2, ('P','I'):-3, ('P','L'):-3, ('P','V'):-2, ('P','W'):-4, ('P','Y'):-3, ('P','F'):-4, ('P','-'):-1.53,
    ('D','C'):-3, ('D','S'): 0, ('D','T'):-1, ('D','A'):-2, ('D','G'):-1, ('D','P'):-1, ('D','D'): 6, ('D','E'): 2, ('D','Q'): 0, ('D','N'): 1, ('D','H'):-1, ('D','R'):-2, ('D','K'):-1, ('D','M'):-3, ('D','I'):-3, ('D','L'):-4, ('D','V'):-3, ('D','W'):-4, ('D','Y'):-3, ('D','F'):-3, ('D','-'):-1.53,
    ('E','C'):-4, ('E','S'): 0, ('E','T'):-1, ('E','A'):-1, ('E','G'):-2, ('E','P'):-1, ('E','D'): 2, ('E','E'): 5, ('E','Q'): 2, ('E','N'): 0, ('E','H'): 0, ('E','R'): 0, ('E','K'): 1, ('E','M'):-2, ('E','I'):-3, ('E','L'):-3, ('E','V'):-2, ('E','W'):-3, ('E','Y'):-2, ('E','F'):-3, ('E','-'):-1.53,       
    ('Q','C'):-3, ('Q','S'): 0, ('Q','T'):-1, ('Q','A'):-1, ('Q','G'):-2, ('Q','P'):-1, ('Q','D'): 0, ('Q','E'): 2, ('Q','Q'): 5, ('Q','N'): 0, ('Q','H'): 0, ('Q','R'): 1, ('Q','K'): 1, ('Q','M'): 0, ('Q','I'):-3, ('Q','L'):-2, ('Q','V'):-2, ('Q','W'):-2, ('Q','Y'):-1, ('Q','F'):-3, ('Q','-'):-1.53,
    ('N','C'):-3, ('N','S'): 1, ('N','T'): 0, ('N','A'):-2, ('N','G'): 0, ('N','P'):-2, ('N','D'): 1, ('N','E'): 0, ('N','Q'): 0, ('N','N'): 6, ('N','H'): 1, ('N','R'): 0, ('N','K'): 0, ('N','M'):-2, ('N','I'):-3, ('N','L'):-3, ('N','V'):-3, ('N','W'):-4, ('N','Y'):-2, ('N','F'):-3, ('N','-'):-1.53,
    ('H','C'):-3, ('H','S'):-1, ('H','T'):-2, ('H','A'):-2, ('H','G'):-2, ('H','P'):-2, ('H','D'):-1, ('H','E'): 0, ('H','Q'): 0, ('H','N'): 1, ('H','H'): 8, ('H','R'): 0, ('H','K'):-1, ('H','M'):-2, ('H','I'):-3, ('H','L'):-3, ('H','V'):-3, ('H','W'):-2, ('H','Y'): 2, ('H','F'):-1, ('H','-'):-1.53,
    ('R','C'):-3, ('R','S'):-1, ('R','T'):-1, ('R','A'):-1, ('R','G'):-2, ('R','P'):-2, ('R','D'):-2, ('R','E'): 0, ('R','Q'): 1, ('R','N'): 0, ('R','H'): 0, ('R','R'): 5, ('R','K'): 2, ('R','M'):-1, ('R','I'):-3, ('R','L'):-2, ('R','V'):-3, ('R','W'):-3, ('R','Y'):-2, ('R','F'):-3, ('R','-'):-1.53,
    ('K','C'):-3, ('K','S'): 0, ('K','T'):-1, ('K','A'):-1, ('K','G'):-2, ('K','P'):-1, ('K','D'):-1, ('K','E'): 1, ('K','Q'): 1, ('K','N'): 0, ('K','H'):-1, ('K','R'): 2, ('K','K'): 5, ('K','M'):-1, ('K','I'): 3, ('K','L'):-2, ('K','V'):-2, ('K','W'):-3, ('K','Y'):-2, ('K','F'):-3, ('K','-'):-1.53,
    ('M','C'):-1, ('M','S'):-1, ('M','T'):-1, ('M','A'):-1, ('M','G'):-3, ('M','P'):-2, ('M','D'):-3, ('M','E'):-2, ('M','Q'): 0, ('M','N'):-2, ('M','H'):-2, ('M','R'):-1, ('M','K'):-1, ('M','M'): 5, ('M','I'): 1, ('M','L'): 2, ('M','V'): 1, ('M','W'):-1, ('M','Y'):-1, ('M','F'): 0, ('M','-'):-1.53,
    ('I','C'):-1, ('I','S'):-2, ('I','T'):-1, ('I','A'):-1, ('I','G'):-4, ('I','P'):-3, ('I','D'):-3, ('I','E'):-3, ('I','Q'):-3, ('I','N'):-3, ('I','H'):-3, ('I','R'):-3, ('I','K'):-3, ('I','M'): 1, ('I','I'): 4, ('I','L'): 2, ('I','V'): 3, ('I','W'):-3, ('I','Y'):-1, ('I','F'): 0, ('I','-'):-1.53,
    ('L','C'):-1, ('L','S'):-2, ('L','T'):-1, ('L','A'):-1, ('L','G'):-4, ('L','P'):-3, ('L','D'):-4, ('L','E'):-3, ('L','Q'):-2, ('L','N'):-3, ('L','H'):-3, ('L','R'):-2, ('L','K'):-2, ('L','M'): 2, ('L','I'): 2, ('L','L'): 4, ('L','V'): 1, ('L','W'):-2, ('L','Y'):-1, ('L','F'): 0, ('L','-'):-1.53,
    ('V','C'):-1, ('V','S'):-2, ('V','T'): 0, ('V','A'):-1, ('V','G'):-3, ('V','P'):-2, ('V','D'):-3, ('V','E'):-2, ('V','Q'):-2, ('V','N'):-3, ('V','H'):-3, ('V','R'):-3, ('V','K'):-2, ('V','M'): 1, ('V','I'): 3, ('V','L'): 1, ('V','V'): 4, ('V','W'):-3, ('V','Y'):-1, ('V','F'):-1, ('V','-'):-1.53,
    ('W','C'):-2, ('W','S'):-3, ('W','T'):-2, ('W','A'):-3, ('W','G'):-2, ('W','P'):-4, ('W','D'):-4, ('W','E'):-3, ('W','Q'):-2, ('W','N'):-4, ('W','H'):-2, ('W','R'):-3, ('W','K'):-3, ('W','M'):-1, ('W','I'):-3, ('W','L'):-2, ('W','V'):-3, ('W','W'):11, ('W','Y'): 2, ('W','F'): 1, ('W','-'):-1.53,
    ('Y','C'):-2, ('Y','S'):-2, ('Y','T'):-2, ('Y','A'):-2, ('Y','G'):-3, ('Y','P'):-3, ('Y','D'):-3, ('Y','E'):-2, ('Y','Q'):-1, ('Y','N'):-2, ('Y','H'): 2, ('Y','R'):-2, ('Y','K'):-2, ('Y','M'):-1, ('Y','I'):-1, ('Y','L'):-1, ('Y','V'):-1, ('Y','W'): 2, ('Y','Y'): 7, ('Y','F'): 3, ('Y','-'):-1.53,
    ('F','C'):-2, ('F','S'):-2, ('F','T'):-2, ('F','A'):-2, ('F','G'):-3, ('F','P'):-4, ('F','D'):-3, ('F','E'):-3, ('F','Q'):-3, ('F','N'):-3, ('F','H'):-1, ('F','R'):-3, ('F','K'):-3, ('F','M'): 0, ('F','I'): 0, ('F','L'): 0, ('F','V'):-1, ('F','W'): 1, ('F','Y'): 3, ('F','F'): 6, ('F','-'):-1.53,
    ('-','C'):-1.53,('-','S'):-1.53,('-','T'):-1.53,('-','A'):-1.53,('-','G'):-1.53,('-','P'):-1.53,('-','D'):-1.53,('-','E'):-1.53,('-','Q'):-1.53,('-','N'):-1.53,('-','H'):-1.53,('-','R'):-1.53,('-','K'):-1.53,('-','M'):-1.53,('-','I'):-1.53,('-','L'):-1.53,('-','V'):-1.53,('-','W'):-1.53,('-','Y'):-1.53,('-','F'):-1.53,('-','-'):-1.53,
    }

In [31]:
# pick up template that has the most matching with the others

def align_sameLen(template,seq_list):
    # align 1 template vs a list of seq
    SCORES = {}
    for seq in seq_list:
        score, ideal_score = 0,0
        for t,s in zip(list(template),list(seq)):
            score += BLOSUM62[(t,s)]
            ideal_score += BLOSUM62[(t,t)] 
        score = score / ideal_score
        SCORES[seq] = score
    return SCORES

def align_blosum62(template,seq_list):
    max_score = 0
    for seq in seq_list:
        score, ideal_score = 0,0  
        s_mod=''
        for t,s in zip(list(template),list(seq)):
            score += BLOSUM62[(t,s)]
            ideal_score += BLOSUM62[(t,t)]  # score when all res in the template match
            if t==s: s_mod += s
            else: s_mod += '-'
        score = score / ideal_score
        max_score = max(max_score,score)
        #print(f'template:{template} seq: {seq} ideal_score: {ideal_score}/{ideal_score}=1 current_score: {score} max: {max_score} \n{template}\n{s_mod}')
    return max_score

def score_blosum62(template,seq):
    score, ideal_score = 0,0  
    for t,s in zip(list(template.upper()),list(seq.upper())):
        try:
            score += BLOSUM62[(t,s)]
            ideal_score += BLOSUM62[(t,t)]  # score when all res in the template match
        except:
            print('ERROR:',template,'---',seq)
    score = score / ideal_score
    return score


# MAFFT alignment
def mafft_pairwise(template,seq_list):
    aligned = []
    max_score = 0
    
    for seq in seq_list:
        with open('pair.in','w') as fo:
            fo.write(f'>template\n{template}\n')
            fo.write(f'>seq\n{seq}\n')
        
        #os.system(f'/home/chinh/miniforge3/bin/mafft --maxiterate 100 --localpair --quiet {cwd}/pair.in > {cwd}/pair.out')
        os.system(f'/home/chinh/miniforge3/bin/mafft --maxiterate 100 --globalpair --quiet pair.in > pair.out')
        
        tem = []
        with open('pair.out') as lines:
            for line in [line for line in lines if '>' not in line]:
                tem.append(line.strip())
        (template,seq) = tem
        score = score_blosum62(template,seq)
        max_score = max(max_score,score)
        aligned.append((seq,score))
        #print(f'template : {template}\nsequence:  {seq} => {score}')
    return max_score, aligned

def template_pickup(cdr_list):
    ALIGNED, SCORE = {}, {}
    for i,template in enumerate(cdr_list):
        seq_list = cdr_list[:i] + cdr_list[i+1:]  # remaining sequences
        #ALIGNED[template] = align_blosum62(template,seq_list)
        max_score, aligned = mafft_pairwise(template,seq_list)
        SCORE[template] = max_score
        ALIGNED[template] = aligned

        print(f'{i}\n{template}')
        for s in aligned:
            print(s[0],s[1])

    print('aligned:',ALIGNED,'\nScores:',SCORE)
    sel_template,_ = [(k,v) for k,v in SCORE.items() if v == max(SCORE.values())][0]
    return sel_template


def mafft_MSA(cdr_list,fname):
    ALIGNED = {}
    max_score = -100
    with open(f'{fname}_msa.in','w') as fo:
        for i,seq in enumerate(cdr_list):
            fo.write(f'>{i}#\n{seq}\n')
        
    try: 
        #os.system(f'/home/chinh/miniforge3/bin/mafft --maxiterate 1000 --globalpair --inputorder --clustalout --quiet {fname}_msa.in > {fname}_msa.out')
        os.system(f'/home/chinh/miniforge3/bin/mafft --maxiterate 1000 --globalpair --inputorder --clustalout --quiet {fname}_msa.in > {fname}_msa.out')  # imac
        #os.system(f'/Users/chinhsutran/miniforge3/bin/mafft --maxiterate 1000 --globalpair --inputorder --clustalout --quiet {fname}_msa.in > {fname}_msa.out')  # imac
        
        with open(f'{fname}_msa.out') as lines:
            for line in [line for line in lines if '#' in line]:
                aligned_seq = (line.split('#')[1]).strip()
                seq_id = line.split('#')[0]
                max_len = len(aligned_seq)
                try:
                    score = score_blosum62('A'*max_len,aligned_seq)*10 / max_len     # using AAAA...AA as reference to calculate scores
                    ALIGNED[int(seq_id)] = score
                    max_score = max(max_score,score)
                except:
                    print(f'aligned_seq: {aligned_seq},seq_id:{seq_id},max_len:{max_len},max_score:{max_score},score:{score}')
        
        #print(ALIGNED)
        sel_template = [cdr_list[int(k)] for k,v in ALIGNED.items() if v == max(ALIGNED.values())]
    except:
        sel_template = 'none'
        
    return max_score, sel_template, ALIGNED



In [42]:
# test alignment
cdr_list = ['EKDNYATGAWFAY','EGHYGYDGYAMDY','EGHYGYDGYAMDY','KGEFYYGYDGFVY','HMGYNDYLAWFAY','ERGDYSNFWYFDL','EKDNYATGAWFAY']
#template_pickup(['EKDNYATGAWFAY','EGHYGYDGYAMDY','EGHYGYDGYAMDY','KGEFYYGYDGFVY','HMGYNDYLAWFAY','ERGDYSNFWYFDL','EKDNYATGAWFAY'])
mafft_MSA(cdr_list,'h3_seq')

aligned_seq: ----EKDNYATGAWFAY,seq_id:0,max_len:17,max_score:-0.007889273356401385,score:-0.007889273356401385
aligned_seq: EG--HYGYDGYA--MDY,seq_id:1,max_len:17,max_score:-0.007889273356401385,score:-0.015674740484429062
aligned_seq: EG--HYGYDGYA--MDY,seq_id:2,max_len:17,max_score:-0.007889273356401385,score:-0.015674740484429062
aligned_seq: KGEFYYGYDGFV----Y,seq_id:3,max_len:17,max_score:-0.007889273356401385,score:-0.020000000000000004
aligned_seq: ----HMGYNDYLAWFAY,seq_id:4,max_len:17,max_score:-0.007889273356401385,score:-0.01480968858131488
aligned_seq: ----ERGDYSNFWYFDL,seq_id:5,max_len:17,max_score:-0.007889273356401385,score:-0.021730103806228376
aligned_seq: ----EKDNYATGAWFAY,seq_id:6,max_len:17,max_score:-0.007889273356401385,score:-0.007889273356401385


(-0.007889273356401385,
 ['EKDNYATGAWFAY', 'EKDNYATGAWFAY'],
 {0: -0.007889273356401385,
  1: -0.015674740484429062,
  2: -0.015674740484429062,
  3: -0.020000000000000004,
  4: -0.01480968858131488,
  5: -0.021730103806228376,
  6: -0.007889273356401385})

CLUSTER THOSE PDBS WITH THE SAME LENGTH OF H1,H2,H3,...,L3 INTO GROUP TO FIND THE REF CDR

In [2]:
os_path = '/media/chinh/4tb/data'
#os_path = '/Users/chinh'
# os_path = "/Volumes/HOME_4TB/CTTSsyno"   # iMac home

#cwd = f'{os_path}/7_mafft'
#cwd = f'/Volumes/HOME_4TB/7_mafft'
#os.chdir(cwd)

In [4]:
#pdb_ff = f'{os_path}/Synology_AMPM/Collab_Nhan/data/FINAL_datasets/pdb4ML_all_notRelax.csv'
pdb_ff = f'{os_path}/Synology_AMPM/Nhan/data/FINAL_datasets/pdb4ML_all_notRelax_03Feb25.csv'   # imac
pdb_df = pd.read_csv(pdb_ff)
pdb_df.columns

Index(['pdbId', 'VH_fam', 'VL_fam', 'H1_len', 'H2_len', 'H3_len', 'L1_len',
       'L2_len', 'L3_len', 'H1_seq', 'H1_pos', 'H2_seq', 'H2_pos', 'H3_seq',
       'H3_pos', 'L1_seq', 'L1_pos', 'L2_seq', 'L2_pos', 'L3_seq', 'L3_pos',
       'batch', 'H3_score', 'L1_score', 'H3_template', 'L1_template',
       'H3-L1_cluster', 'FWL1_seq', 'FWL2_seq', 'FWL3_seq', 'FWL4_seq',
       'FWH1_seq', 'FWH2_seq', 'FWH3_seq', 'FWH4_seq'],
      dtype='object')

In [4]:
CDRs_df = pdb_df[['pdbId','VH_fam', 'VL_fam','H1_len','H2_len','H3_len','L1_len','L2_len','L3_len','H1_seq','H2_seq','H3_seq','L1_seq','L2_seq','L3_seq']]
CDRs_df

,pdbId,VH_fam,VL_fam,H1_len,H2_len,H3_len,L1_len,L2_len,L3_len,H1_seq,H2_seq,H3_seq,L1_seq,L2_seq,L3_seq
0,1adq_HLA,IGHV03,IGLV03,7,6,14,11,7,11,GFTFDDY,SWNTGT,TRSYVVAAEYYFHY,GGNNIGSKSVH,DDSDRPP,QVWDSSSDHAV
1,1ahw_BAC,IGHV14,IGKV14,7,6,8,11,7,9,GFNIKDY,DPENGN,DNSYYFDY,KASQDIRKYLN,YATSLAD,LQHGESPYT
2,1ahw_EDF,IGHV14,IGKV14,7,6,8,11,7,9,GFNIKDY,DPENGN,DNSYYFDY,KASQDIRKYLN,YATSLAD,LQHGESPYT
3,1bj1_HLW,IGHV07,IGKV01,7,6,14,11,7,9,GYTFTNY,NTYTGE,YPHYYGSSHWYFDV,SASQDISNYLN,FTSSLHS,QQYSTVPWT
4,1bj1_KJV,IGHV07,IGKV01,7,6,14,11,7,9,GYTFTNY,NTYTGE,YPHYYGSSHWYFDV,SASQDISNYLN,FTSSLHS,QQYSTVPWT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,7vux_HLA,IGHV05,IGKV03,7,6,8,11,7,9,GFAFSSY,SGGGRY,PYGGYFDV,RASQSISNFLH,YASQSIS,QQSNSWPHT
2830,7vyr_ABC,IGHV01,IGLV03,7,6,9,11,7,10,GGTFGNW,DPSRGE,ETEQGGMDV,GGNNIGRKSVH,DDSDRPS,QVYDLFSDYV
2831,7vyr_HLR,IGHV01,IGLV03,7,6,9,11,7,10,GGTFGNW,DPSRGE,ETEQGGMDV,GGNNIGRKSVH,DDSDRPS,QVYDLFSDYV
2832,7vyt_BCA,IGHV01,IGKV03,7,6,12,12,7,10,GYTFTSY,NPSGGS,RSGSGWFGALDY,RASQSVSSSYLA,GASSRAT,QQYGSSPGGT


In [11]:
VHVL_fams = CDRs_df[['VH_fam', 'VL_fam','H3_len','L1_len']].drop_duplicates().values
print(VHVL_fams.shape)
VHVL_fams

(573, 4)


array([['IGHV03', 'IGLV03', 14, 11],
       ['IGHV14', 'IGKV14', 8, 11],
       ['IGHV07', 'IGKV01', 14, 11],
       ...,
       ['IGHV02', 'IGLV02', 10, 14],
       ['IGHV05', 'IGKV03', 8, 11],
       ['IGHV01', 'IGKV03', 12, 12]], dtype=object)

In [ ]:
print(VHVL_fams.tolist())
with open(f'{os_path}/Synology_across/projects/SabDab_Ab-Ag/VHVLh3l1_clusters.csv','w') as fo:
    fo.write('#VH_fam,VL_fam,H3_len,L1_len')
    for i in sorted(VHVL_fams.tolist()):
        for j in i: 
            fo.write(f'{j},')
        fo.write('\n')


[['IGHV03', 'IGLV03', 14, 11], ['IGHV14', 'IGKV14', 8, 11], ['IGHV07', 'IGKV01', 14, 11], ['IGHV04', 'IGKV01', 8, 11], ['IGHV03', 'IGKV01', 12, 11], ['IGHV01', 'IGKV03', 10, 15], ['IGHV02', 'IGKV10', 7, 11], ['IGHV02', 'IGKV10', 14, 11], ['IGHV01', 'IGKV03', 19, 11], ['IGHV05', 'IGKV03', 12, 15], ['IGHV01', 'IGKV3D', 9, 15], ['IGHV01', 'IGKV03', 8, 12], ['IGHV03', 'IGKV01', 8, 11], ['IGHV08', 'IGKV13', 12, 11], ['IGHV01', 'IGKV03', 10, 11], ['IGHV03', 'IGKV01', 11, 11], ['IGHV01', 'IGKV03', 8, 15], ['IGHV05', 'IGKV06', 13, 11], ['IGHV03', 'IGKV01', 10, 11], ['IGHV01', 'IGKV04', 10, 10], ['IGHV05', 'IGKV12', 13, 11], ['IGHV05', 'IGKV06', 11, 11], ['IGHV01', 'IGKV1D', 8, 11], ['IGHV06', 'IGKV04', 5, 10], ['IGHV01', 'IGKV10', 11, 11], ['IGHV02', 'IGKV05', 11, 11], ['IGHV03', 'IGKV01', 20, 11], ['IGHV09', 'IGKV19', 9, 11], ['IGHV01', 'IGKV01', 10, 11], ['IGHV01', 'IGLV03', 9, 11], ['IGHV04', 'IGKV03', 5, 11], ['IGHV01', 'IGKV04', 9, 10], ['IGHV03', 'IGKV6D', 12, 10], ['unknown', 'unknown',

In [17]:
VHVL_GROUPS = {'pdbId':[],'H3_score':[],'L1_score':[],'H3_template':[],'L1_template':[],'H3_cluster':[],'L1_cluster':[]}
for i,vhvl in enumerate(VHVL_fams):
    (vh,vl,h3,l1) = vhvl
    tem_df = CDRs_df.loc[(CDRs_df['VH_fam'] == vh) & (CDRs_df['VL_fam'] == vl) & (CDRs_df['H3_len'] == h3) & (CDRs_df['L1_len'] == l1)]
    #print(vhvl,'\n',tem_df)

    os.system(f'mkdir -p {cwd}/{vh}_{vl}_{h3}_{l1}')
    os.chdir(f'{cwd}/{vh}_{vl}_{h3}_{l1}')

    for pdbId in tem_df['pdbId'].values:
        VHVL_GROUPS['pdbId'].append(pdbId)  # h1,h2,h3,l1,l2,l3 scores in this order

    #for seqs in ['H1_seq','H2_seq','H3_seq','L1_seq','L2_seq','L3_seq']:
    for cdr_seq in ['H3_seq','L1_seq']:
        sel_template = []
        cdr_list = tem_df[cdr_seq].values
        try:
            max_score, sel_template, ALIGNED = mafft_MSA(cdr_list,cdr_seq[:2])
                # ALIGNED = {0:score, 1:score, etc.} with keys = indices of cdr_list OR pdb_list
            for i in range(len(cdr_list)):
                VHVL_GROUPS[f'{cdr_seq[:2]}_score'].append(ALIGNED[i])
                VHVL_GROUPS[f'{cdr_seq[:2]}_template'].append(sel_template[0])
                VHVL_GROUPS[f'{cdr_seq[:2]}_cluster'].append(f'{vh}_{vl}_{h3}_{l1}')
        except:
            print('mafft ERROR due to so many Ns:',cdr_seq,cdr_list)
            #print(vhvl,'\n',tem_df[cdr_seq])

            for seq in tem_df[cdr_seq].values:
                if seq.count('N') >= len(seq)/3: pass
                else: sel_template.append(seq)
            #break

        #print(f'{cdr_seq}, sel_template: {set(sel_template)}')

print('\nVHVLh3l1 groups:',VHVL_GROUPS)
for k,v in VHVL_GROUPS.items():
    print(k,len(VHVL_GROUPS[k]))
df = pd.DataFrame(VHVL_GROUPS)
print(df)

df.to_csv(f'{cwd}/pdb4ML_all_notRelax_VHVLh3l1_profile.csv')



VHVLh3l1 groups: {'pdbId': ['1adq_HLA', '4dag_HLA', '6b0s_HLC', '6wtu_BCA', '6wtu_EFD', '6wtu_HIG', '6wtu_KLJ', '6wtv_BCA', '6wtv_EFD', '6wtv_HIG', '6wtv_KLJ', '1ahw_BAC', '1ahw_EDF', '1bj1_HLW', '1bj1_KJV', '1cz8_HLW', '1cz8_YXV', '1bvk_BAC', '1bvk_EDF', '2jix_DGE', '2jix_HLE', '6i04_CDB', '6i04_HLA', '6r2s_BAC', '7bq5_CDB', '7bq5_HLA', '1dee_DCG', '1dee_FEH', '1hez_BAE', '1hez_DCE', '2fjh_BAW', '2fjh_HLV', '2r56_HLA', '2r56_IMB', '3eoa_BAJ', '3eoa_HLI', '3eob_BAJ', '3eob_HLI', '3p0y_HLA', '3p11_HLA', '3wd5_HLA', '4lmq_EIF', '4lmq_HLD', '4zs6_CDB', '4zs6_HLA', '5d1z_BAI', '5d1z_GHJ', '5vl7_HLB', '6x4t_BEC', '6x4t_DFA', '6zcz_HLE', '6zdh_DFB', '6zdh_GIC', '6zdh_HLA', '6zer_BCA', '6zer_FGD', '6zer_HLE', '7bel_ABR', '7bel_CDX', '7jx3_HLR', '7kfv_CDB', '7kfv_FGE', '7kfv_HLA', '7kfw_CDB', '7kfw_FGE', '7kfw_HLA', '7kfx_HLA', '7kfy_HLA', '7l0n_HLR', '7l0n_MNS', '7nx6_HLE', '7nx7_HLE', '7nx8_HLE', '7nx9_HLE', '7nxa_HLE', '7nxb_HLE', '7ora_ABR', '7ora_DEC', '7pry_ABI', '7pry_FGE', '7qnw_HLE',

In [12]:
cdrLen_groups = CDRs_df[['H1_len','H2_len','H3_len','L1_len','L2_len','L3_len']].drop_duplicates().values
print(cdrLen_groups.shape)

(454, 6)


In [14]:
CDR_GROUPS = {}
for i,g in enumerate(cdrLen_groups):
    (h1,h2,h3,l1,l2,l3) = g
    #tem_df = CDRs_df.loc[(CDRs_df['H1_len'] == h1) & (CDRs_df['H2_len'] == h2) & (CDRs_df['H3_len'] == h3) & (CDRs_df['L1_len'] == l1) & (CDRs_df['L2_len'] == l2) & (CDRs_df['L3_len'] == l3)]
    tem_df = CDRs_df.loc[(CDRs_df['H1_len'] == h1) & (CDRs_df['H3_len'] == h3) & (CDRs_df['L1_len'] == l1) ]
    #tem_df = CDRs_df.loc[(CDRs_df['H3_len'] == h3)]
    CDR_GROUPS[f'{g}'] = tem_df
    #print(g,'\n',tem_df[['pdbId','VH_fam', 'VL_fam','H1_seq','H3_seq','L1_seq']])
#CDR_GROUPS

SCRIPT To test run_model on loops of clusters

In [12]:
# cwd = f'/Volumes/HOME_4TB/7_mafft'
cwd = f'{os_path}/research/SabDab_Ab-Ag/6_MLmodels/1_AgInterface_Nhan'
os.chdir(cwd)

cluster_df = pdb_df[['pdbId','VH_fam', 'VL_fam','H1_len','H2_len','H3_len','L1_len','L2_len','L3_len','H1_seq','H2_seq','H3_seq','L1_seq','L2_seq','L3_seq','H3_score', 'L1_score','H3-L1_cluster']]
# print(cluster_df.head(3))

VHVLH3L1_cluster = sorted(cluster_df[['H3-L1_cluster']].drop_duplicates().values.tolist())
print(len(VHVLH3L1_cluster))
print(VHVLH3L1_cluster)
#'''
with open(f'{cwd}/2025-06-19_clusterH3L1.txt','w') as fo:
    #fo.write('#VH,VL,score_H3,score_L1,lenH1,lenH2,lenH3,lenL1,lenL2,lenL3\n')
    for c in VHVLH3L1_cluster:
        ## IGHV01_IGKV03_9_12
        try:
            #print(c)
            VH,VL,lenH3,lenL1 = c[0].split('_')
            VH = f'VH{int(VH[-2:])}' # VH11
            if 'KV' in VL:
                VL = f'VK{int(VL[4:6])}'
            elif 'LV' in VL:
                VL = f'V{VL[2]}a{int(VL[-2:])}'
            lenH3 = int(lenH3)
            lenL1 = int(lenL1)
            score_H3 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0])]['H3_score'].min()
            score_L1 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3)]['L1_score'].min()
            # pick up remaining lens wrt the selected H3 and L1 scores
            lenH1 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3)]['H1_len'].min()
            lenH2 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3)]['H2_len'].min()
            lenL2 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['L1_score'] == score_L1)]['L2_len'].min()
            lenL3 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['L1_score'] == score_L1)]['L3_len'].min()
            # pick up seq
            seqH1 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3) & (cluster_df['H1_len'] == lenH1)]['H1_seq'].values[0]
            seqH2 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3) & (cluster_df['H2_len'] == lenH2)]['H2_seq'].values[0]
            seqH3 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['H3_score'] == score_H3) & (cluster_df['H3_len'] == lenH3)]['H3_seq'].values[0]
            seqL1 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['L1_score'] == score_L1) & (cluster_df['L1_len'] == lenL1)]['L1_seq'].values[0]
            seqL2 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['L1_score'] == score_L1) & (cluster_df['L2_len'] == lenL2)]['L2_seq'].values[0]
            seqL3 = cluster_df.loc[(cluster_df['H3-L1_cluster'] == c[0]) & (cluster_df['L1_score'] == score_L1) & (cluster_df['L3_len'] == lenL3)]['L3_seq'].values[0]
            # print(VH,VL,score_H3,score_L1,lenH1,lenH2,lenH3,lenL1,lenL2,lenL3)
            # print(seqH1)
            fo.write(f'{VH},{VL},{score_H3},{score_L1},{lenH1},{lenH2},{lenH3},{lenL1},{lenL2},{lenL3},{seqH1},{seqH2},{seqH3},{seqL1},{seqL2},{seqL3}\n')
        except:
            print(c[0],'--pass')
            pass
#'''

573
[['IGHV01_IGKV01_10_11'], ['IGHV01_IGKV01_10_16'], ['IGHV01_IGKV01_10_7'], ['IGHV01_IGKV01_11_11'], ['IGHV01_IGKV01_11_16'], ['IGHV01_IGKV01_12_10'], ['IGHV01_IGKV01_12_11'], ['IGHV01_IGKV01_13_11'], ['IGHV01_IGKV01_13_16'], ['IGHV01_IGKV01_14_11'], ['IGHV01_IGKV01_15_11'], ['IGHV01_IGKV01_15_15'], ['IGHV01_IGKV01_16_11'], ['IGHV01_IGKV01_16_16'], ['IGHV01_IGKV01_17_11'], ['IGHV01_IGKV01_18_11'], ['IGHV01_IGKV01_19_11'], ['IGHV01_IGKV01_20_11'], ['IGHV01_IGKV01_21_11'], ['IGHV01_IGKV01_23_11'], ['IGHV01_IGKV01_25_11'], ['IGHV01_IGKV01_5_11'], ['IGHV01_IGKV01_6_11'], ['IGHV01_IGKV01_7_10'], ['IGHV01_IGKV01_7_11'], ['IGHV01_IGKV01_7_16'], ['IGHV01_IGKV01_8_11'], ['IGHV01_IGKV01_8_16'], ['IGHV01_IGKV01_9_10'], ['IGHV01_IGKV02_12_16'], ['IGHV01_IGKV02_13_16'], ['IGHV01_IGKV02_16_16'], ['IGHV01_IGKV02_18_16'], ['IGHV01_IGKV02_19_16'], ['IGHV01_IGKV02_26_16'], ['IGHV01_IGKV02_9_16'], ['IGHV01_IGKV03_10_11'], ['IGHV01_IGKV03_10_12'], ['IGHV01_IGKV03_10_15'], ['IGHV01_IGKV03_11_12'], ['IGH

Bad pipe message: %s [b'Y|\x1c\xc3\x01\x88\xe3\x1f\xe4\xb93\x16z\xd0\xa6\xd0\xb2\xf3\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00', b'\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00']
Bad pipe message: %s [b'\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)']
Bad pipe message: %s [b'\'\x06\x9f\x02\xfc\xc9FV\xcc\xfck*r\x0b\xb8\xc8\xc8M\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00']
Bad pipe message: %s [b'D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\

ALIGN UNSEEN DATA TO THE DATASET
1. extract (vh_fam,vl_fam,h3_len,l1_len,h3_seq,l1_seq) from unseen data
2. map these unseen_info into classified H3_cluster/L1_cluster to pick up the corresponding template seqs of H3 and L1
3. align the templates and the unseen_seq to calculate the H3/L1_score to complete profiles for the unseen

ML_VHVLh3l1_profile = "pdb4ML_all_notRelax_VHVLh3l1_profile.csv"

In [28]:
cwd = f'{os_path}/research/SabDab_Ab-Ag/7_mafft'
#cwd = f'/Volumes/HOME_4TB/7_mafft'
os.chdir(cwd)

profile_path = "/media/chinh/4tb/data/research/SabDab_Ab-Ag/7_mafft"
model_VHVLh3l1_profile = f'{profile_path}/pdb4ML_all_notRelax_VHVLh3l1_profile.csv'

df_model = pd.read_csv(model_VHVLh3l1_profile)
df_model

,pdbId,H3_score,L1_score,H3_template,L1_template,H3_cluster,L1_cluster
0,1adq_HLA,-0.089286,-0.144628,VLSRASGMPDAFDI,SGDKLGDKYAS,IGHV03_IGLV03_14_11,IGHV03_IGLV03_14_11
1,1ahw_BAC,-0.507812,-0.144628,DNSYYFDY,KASQDIRKYLN,IGHV14_IGKV14_8_11,IGHV14_IGKV14_8_11
2,1ahw_EDF,-0.507812,-0.144628,DNSYYFDY,KASQDIRKYLN,IGHV14_IGKV14_8_11,IGHV14_IGKV14_8_11
3,1bj1_HLW,-0.242347,-0.082645,YPHYYGSSHWYFDV,SASQDISNYLN,IGHV07_IGKV01_14_11,IGHV07_IGKV01_14_11
4,1bj1_KJV,-0.242347,-0.082645,YPHYYGSSHWYFDV,SASQDISNYLN,IGHV07_IGKV01_14_11,IGHV07_IGKV01_14_11
...,...,...,...,...,...,...,...
2829,7vux_HLA,-0.390625,-0.061983,PYGGYFDV,RASQSISNFLH,IGHV05_IGKV03_8_11,IGHV05_IGKV03_8_11
2830,7vyr_ABC,-0.216049,-0.185950,ETEQGGMDV,GGNNIGSKSVH,IGHV01_IGLV03_9_11,IGHV01_IGLV03_9_11
2831,7vyr_HLR,-0.216049,-0.185950,ETEQGGMDV,GGNNIGSKSVH,IGHV01_IGLV03_9_11,IGHV01_IGLV03_9_11
2832,7vyt_BCA,-0.086806,0.121528,RSGSGWFGALDY,RASQSVSSSYLA,IGHV01_IGKV03_12_12,IGHV01_IGKV03_12_12


In [9]:
H3L1_clusters = df_model['H3_cluster'].drop_duplicates().values
print(f'H3L1_clusters_len: {len(H3L1_clusters)}')


H3L1_clusters_len: 573


In [17]:
unseen_path = "/media/chinh/4tb/data/Synology_AMPM/Collab_Nhan/data/FINAL_datasets"
unseen_VHVLh3l1_profile = f'{unseen_path}/unprocessed_covid.csv'

df_unseen = pd.read_csv(unseen_VHVLh3l1_profile)
df_unseen = df_unseen.loc[(df_unseen['status'] == 'ready for data')][['pdbID','VH_fam','VL_fam','H3_len','L1_len','H3_seq','L1_seq']]
df_unseen

,pdbID,VH_fam,VL_fam,H3_len,L1_len,H3_seq,L1_seq
0,7e39_CBA,IGHV03,IGKV01,9,11,ETVSYGMDV,RARQGISNYLA
1,7e86_ABC,IGHV03,IGKV01,9,11,DAQNYGMDV,RASQSISSYLN
2,7e88_ABC,IGHV03,IGKV01,8,11,DRGLVSDY,QASQDINKYLN
3,7e88_DEF,IGHV03,IGKV01,8,11,DRGLVSDY,QASQDINKYLN
4,7e88_GHI,IGHV03,IGKV01,8,11,DRGLVSDY,QASQDINKYLN
...,...,...,...,...,...,...,...
591,8j1t_HKF,IGHV03,IGKV01,9,11,DHSGHALDI,RASQGISSYLA
592,8j1v_CDB,IGHV03,IGKV01,9,11,DHSGHALDI,RASQGISSYLA
593,8j1v_FGE,IGHV03,IGKV01,9,11,DHSGHALDI,RASQGISSYLA
594,8j1v_HKA,IGHV03,IGKV01,9,11,DHSGHALDI,RASQGISSYLA


In [39]:
unseen_DICT = {'pdbId':[],'H3_score':[],'L1_score':[],'VHVLh3l1':[],'H3_template':[],'L1_template':[],'align_status':[]}

cnt=0
for pdb in df_unseen['pdbID'].values:
    #print('\n',pdb)
    VH,VL,h3_len,l1_len,h3_seq,l1_seq = df_unseen.loc[df_unseen['pdbID']==pdb].values.flatten().tolist()[1:]

    unseen_DICT['pdbId'].append(pdb)
    unseen_DICT['VHVLh3l1'].append(f'{VH}_{VL}_{h3_len}_{l1_len}')


    unseen_tem = f'{VH}_{VL}_{h3_len}_{l1_len}'
    try:
        h3_template,l1_template = df_model.loc[df_model['H3_cluster'] == unseen_tem].values.flatten().tolist()[3:5]
        ## align
        _, _, H3_dict = mafft_MSA([h3_template,h3_seq],'H3')
        H3_score = H3_dict[1]
        unseen_DICT['H3_score'].append(H3_score)
        unseen_DICT['H3_template'].append(h3_template)
        #print(h3_template,h3_seq,H3_score)
    
        _, _, L1_dict = mafft_MSA([l1_template,l1_seq],'L1')
        L1_score = L1_dict[1]
        unseen_DICT['L1_score'].append(L1_score)
        unseen_DICT['L1_template'].append(l1_template)

        unseen_DICT['align_status'].append('aligned')
        cnt+=1

    except:
        #print(f'ERROR -- {VH}_{VL} not found in model_clusters')
        unseen_DICT['H3_score'].append('nil')
        unseen_DICT['L1_score'].append('nil')
        unseen_DICT['H3_template'].append('NA')
        unseen_DICT['L1_template'].append('NA')
        unseen_DICT['align_status'].append('clusters_notExist')

print(f'{cnt} unseen pdbs aligned -- {len(df_unseen) - cnt}')

df_out = pd.DataFrame(unseen_DICT)
print(df_out)

df_out.to_csv(f'{cwd}/covid_VHVLh3l1_profile.csv')

    

454 unseen pdbs aligned -- 142
        pdbId  H3_score  L1_score             VHVLh3l1    H3_template  \
0    7e39_CBA -0.216049       0.0   IGHV03_IGKV01_9_11      STAAEFFDY   
1    7e86_ABC -0.216049       0.0   IGHV03_IGKV01_9_11      STAAEFFDY   
2    7e88_ABC   -0.3125 -0.165289   IGHV03_IGKV01_8_11       DTAAYFDY   
3    7e88_DEF   -0.3125 -0.165289   IGHV03_IGKV01_8_11       DTAAYFDY   
4    7e88_GHI   -0.3125 -0.165289   IGHV03_IGKV01_8_11       DTAAYFDY   
..        ...       ...       ...                  ...            ...   
591  8j1t_HKF -0.154321  0.103306   IGHV03_IGKV01_9_11      STAAEFFDY   
592  8j1v_CDB -0.154321  0.103306   IGHV03_IGKV01_9_11      STAAEFFDY   
593  8j1v_FGE -0.154321  0.103306   IGHV03_IGKV01_9_11      STAAEFFDY   
594  8j1v_HKA -0.154321  0.103306   IGHV03_IGKV01_9_11      STAAEFFDY   
595  8pq2_HLC  -0.14248 -0.103306  IGHV04_IGKV01_13_11  DRFDVASGSSFDF   

     L1_template align_status  
0    RASQAVSSAVA      aligned  
1    RASQAVSSAVA      aligne